In [1]:
"""fixture"""
import os
import time
import string
import pytest

In [2]:
"""import"""
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
"""data preparation"""
batch_size = 128
num_classes = 10
epochs = 4

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
assert x_train.shape == (60000,28,28,1)
assert x_test.shape == (10000,28,28,1)
assert y_train.shape == (60000, )
assert y_test.shape == (10000, )
assert input_shape == (img_rows, img_cols, 1)

In [4]:
"""prepare x"""
@pytest.mark.timeout(180)
def prepareX(x_train, x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    return x_train,x_test
x_train,x_test = prepareX(x_train,x_test)

In [5]:
"""print x shape"""
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
"""prepare y"""
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

assert y_train.shape == (60000,10)
assert y_test.shape == (10000,10)

In [7]:
"""build model"""
@pytest.mark.timeout(10)
def test_buildModel():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model
model = test_buildModel()

In [8]:
"""compile model"""
@pytest.mark.timeout(10)
def test_compileModel(model):
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model
model = test_compileModel(model)

In [9]:
"""train model"""
@pytest.mark.timeout(600)
def test_trainModel(model):
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        validation_data=(x_test, y_test))
    return model,history
model,history = test_trainModel(model)

In [10]:
"""score model"""
@pytest.mark.timeout(120)
def test_scoreModel(model):
    score = model.evaluate(x_test, y_test, verbose=0)
    print(score)
    return score
score = test_scoreModel(model)
assert score[0] < 0.05
assert score[1] > 0.98

[0.036879136996716258, 0.9879]


In [11]:
"""print score"""
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0368791369967
Test accuracy: 0.9879
